In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''load datasets'''

X_train = np.load('data\X_train.npy')# population in F=1
Y_train = np.load('data\Y_train.npy')#normalized to w_L


X_test = np.load('data\X_test.npy') # use validation set for testing
Y_test = np.load('data\Y_test.npy') # use validation set for testing



In [ ]:
model_cnn = keras.models.Sequential([
    keras.layers.SeparableConv1D(kernel_size=8, activation='relu',padding='SAME',filters=32),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.SeparableConv1D(kernel_size=8, activation='relu',padding='SAME',filters=64),
    keras.layers.SeparableConv1D(kernel_size=8, activation='relu',padding='SAME',filters=64),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.SeparableConv1D(kernel_size=4, activation='relu',padding='SAME',filters=128),
    keras.layers.SeparableConv1D(kernel_size=4, activation='relu',padding='SAME',filters=128),
    keras.layers.Flatten(),
    # keras.layers.Dense(200,activation='relu'),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(3)
    ])

model_cnn.compile(loss="mse", optimizer = keras.optimizers.Adam(1e-3,decay=1e-5),metrics=["mae"])#loss functions are given for two output neurons


In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath="models.hdf5",save_best_only=True, monitor='val_mae')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=500, monitor='val_mae',restore_best_weights=True)



In [ ]:
history_cnn = model_cnn.fit(X_train, Y_train, epochs=1500, validation_data=(X_test, Y_test),batch_size=4,callbacks=[checkpoint_cb,early_stopping_cb])


In [ ]:
h_cnn = history_cnn.history

training_loss = np.array(h_cnn["loss"])
test_loss = np.array(h_cnn["val_loss"])
epoch_count = np.arange(len(training_loss)) + 1

fig, ax = plt.subplots()
ax.plot(epoch_count, np.sqrt(training_loss),label='training')
ax.plot(epoch_count, np.sqrt(test_loss),label='val')
ax.set_xlabel(r'epoch_count',fontsize=12)
ax.set_ylabel(r'sqrt(Loss)',fontsize=12)
ax.legend(loc=0)
ax.set_ylim(0, 0.02)

F = model_cnn.predict(X_test)